In [1]:
import torch.nn as nn
import torch.optim as optim
import shutil
import os
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd
import pickle
from fair_loss import FairLoss
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import io
from torchmetrics.classification import BinaryFairness
from fairlearn.metrics import demographic_parity_ratio
from fairlearn.metrics import demographic_parity_difference

from sklearn.metrics import f1_score, balanced_accuracy_score,recall_score

# from datasets import ADULT, German, Lawschool, HealthHeritage
# from defenses import dp_defense


In [2]:
import torch
print(torch.__version__)


2.3.1+cu121


In [3]:
class LinReLU(nn.Module):

    """
    A linear layer followed by a ReLU activation layer.
    """    
    
    def __init__(self, in_size, out_size):
        super(LinReLU, self).__init__()      
        linear = nn.Linear(in_size, out_size)
        ReLU = nn.ReLU()
        # self.Dropout = nn.Dropout(0.25)
        self.layers = nn.Sequential(linear, ReLU)

    def reset_parameters(self):
        self.layers[0].reset_parameters()
        return self

    def forward(self, x):
        x = self.layers(x)
        return x

class FullyConnected(nn.Module):
    """
    A simple fully connected neural network with ReLU activations.
    """
    def __init__(self, input_size, layout):

        super(FullyConnected, self).__init__()
        layers = [nn.Flatten()]  # does not play any role, but makes the code neater
        prev_fc_size = input_size
        for i, fc_size in enumerate(layout):
            if i + 1 < len(layout):
                layers += [LinReLU(prev_fc_size, fc_size)]
            else:
                layers += [nn.Linear(prev_fc_size, 1), nn.Sigmoid()]
                # layers += [nn.Linear(prev_fc_size, fc_size)]
            prev_fc_size = fc_size
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return x

In [7]:
# state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
#                "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
#                "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
#                "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
#                "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# state_codes=["WW","WM","BM","BW"]
state_codes=["male","female","white","black"]

In [8]:
client_data_dir='50_clients_data/client_subG_processed/'

layout = [100, 100, 2]
batch_size = 32
num_epochs = 5  
input_dim = 10

model = FullyConnected(input_dim, layout)
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01) 

# Normal training

In [9]:
# 0 male
# 1 female
# 1 white
# 2 black

In [10]:
model.load_state_dict(torch.load('50_clients_data/clients_trained_model/AL.pth'))
# model.load_state_dict(torch.load('50_clients_data/client_DP_trained_model/AL.pth'))
# model.load_state_dict(torch.load('50_clients_data/clients_fair_trained_model/AL.pth'))
# model.load_state_dict(torch.load('50_clients_data/clients_DP_Fair_trained_model/AL.pth'))

<All keys matched successfully>

In [11]:
# for state_code in state_codes:
    
#     # print(state_code)    
#     state_name=state_code
    
#     with open(client_data_dir+f'{state_name}.pkl', 'rb') as f:
#         data  = pickle.load(f)


# test_data = data
# len(data),len(data)*32


In [12]:
    #  tensor(1.0448) convert to 1 ------Female
    #  tensor(-0.9571) convert to 0----Male

In [13]:
50*32

1600

In [15]:
for state_code in state_codes:
    print(state_code)

    with open(client_data_dir + f'{state_code}.pkl', 'rb') as f:
        test_data = pickle.load(f)

    fairness_score = []
    fair_ratio = []
    parity_diff = []
    temp_out = []
    temp_lab = []
    temp_group = []

    model.eval()
    with torch.no_grad():
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        threshold = 0.0
        all_labels = []
        all_predicted_classes = []

        for i, (inputs, labels) in enumerate(test_data):
            if i >= 50: #50*32 =1600 points:
                print(i)
                break
            labels = labels.unsqueeze(1).float()
            temp_lab.append(labels)

            #=====================Need to check=====================#
            groups = inputs[:, 8].long()
            groups = (groups > threshold).long()
            temp_group.append(groups)
            #======================================================#

            outputs = model(inputs)
            # temp_out.append(outputs)

            metric = BinaryFairness(2)
            temp_fairness_score = metric(outputs, labels, groups)
            fairness_score.append(temp_fairness_score)

            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item()

            predicted_classes = (outputs > 0.5).float()
            temp_out.append(predicted_classes)

            val_correct += (predicted_classes == labels).sum().item()
            val_total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_predicted_classes.extend(predicted_classes.cpu().numpy())

        val_epoch_loss = val_running_loss / len(test_data)
        val_accuracy = val_correct / val_total

        all_labels = np.array(all_labels)
        all_predicted_classes = np.array(all_predicted_classes)
        f1 = f1_score(all_labels, all_predicted_classes)
        balanced_acc = balanced_accuracy_score(all_labels, all_predicted_classes)
        tpr = recall_score(all_labels, all_predicted_classes)

        print(val_epoch_loss, ":", val_accuracy, ":", f1, ":", balanced_acc, ":", tpr)


male
50
0.07884532413162744 : 0.733125 : 0.8124725516029864 : 0.6708755419143813 : 0.9371833839918946
female
50
0.071832009873832 : 0.794375 : 0.8671780379491321 : 0.6901417158749754 : 0.907861369399831
white
50
0.07985490950913475 : 0.735 : 0.8130511463844797 : 0.6722505949356934 : 0.9183266932270916
black
0.4222907843068242 : 0.786 : 0.8635204081632653 : 0.6728429340385641 : 0.8896189224704336


In [20]:
0.907861369399831/  0.9371833839918946

0.9687126179433873

In [ ]:
# Normal
# min(BW)/max (WM)
#0.9524043710833204

# DP:
# min(BM)/max (WM)
# 0.9480966322402125

# Fair
# min(BW)/max (WM)
# 0.9358371581936287

# DP+Fair
# min(BW)/max (WM)
# 0.931684665747398

In [ ]:

#16000

# Normal
# min(BW)/max (WM)
#0.9355793421769767


# DP:
# min(BM)/max (WM)
# 0.9408830403540616

# Fair
# min(BW)/max (WM)
# 0.9448831280916116

# DP+Fair
# min(BW)/max (WM)
# 0.9409639144847498

In [16]:
fairness_score

[{'DP_0_1': tensor(0.6303), 'EO_0_1': tensor(0.6364)},
 {'DP_0_1': tensor(0.8036), 'EO_0_0': tensor(1.)},
 {'DP_0_1': tensor(0.7647), 'EO_0_1': tensor(0.8667)},
 {'DP_0_1': tensor(0.8974), 'EO_0_1': tensor(0.9028)},
 {'DP_0_1': tensor(0.6272), 'EO_0_1': tensor(0.7273)},
 {'DP_0_1': tensor(0.6842), 'EO_0_1': tensor(0.7059)},
 {'DP_0_1': tensor(0.8000), 'EO_0_1': tensor(0.9778)},
 {'DP_0_1': tensor(0.8235), 'EO_0_1': tensor(0.8182)},
 {'DP_0_1': tensor(0.6429), 'EO_0_1': tensor(0.6154)},
 {'DP_0_1': tensor(0.9600), 'EO_0_1': tensor(0.9519)},
 {'DP_0_1': tensor(0.8086), 'EO_0_1': tensor(0.8250)},
 {'DP_0_1': tensor(0.9821), 'EO_1_0': tensor(0.9890)},
 {'DP_0_1': tensor(0.8824), 'EO_0_0': tensor(1.)},
 {'DP_0_1': tensor(0.7895), 'EO_0_1': tensor(0.8750)},
 {'DP_1_0': tensor(0.9783), 'EO_1_0': tensor(0.9231)},
 {'DP_0_1': tensor(0.7500), 'EO_0_1': tensor(0.7778)},
 {'DP_1_0': tensor(0.8571), 'EO_0_1': tensor(0.9630)},
 {'DP_0_1': tensor(0.8708), 'EO_0_1': tensor(0.9778)},
 {'DP_0_1': tensor

In [17]:
# Initialize sums and counts
dp_0_1_sum, eo_0_1_sum = 0.0, 0.0
dp_1_0_sum, eo_1_0_sum = 0.0, 0.0
dp_0_1_count, eo_0_1_count = 0, 0
dp_1_0_count, eo_1_0_count = 0, 0

dp_0_0_sum, eo_0_0_sum = 0.0, 0.0
dp_0_0_count, eo_0_0_count = 0, 0

# Calculate sums and counts
for entry in fairness_score:
    if 'DP_0_1' in entry:
        dp_0_1_sum += entry['DP_0_1'].item()
        dp_0_1_count += 1
    if 'EO_0_1' in entry:
        eo_0_1_sum += entry['EO_0_1'].item()
        eo_0_1_count += 1
    if 'DP_1_0' in entry:
        dp_1_0_sum += entry['DP_1_0'].item()
        dp_1_0_count += 1
    if 'EO_1_0' in entry:
        eo_1_0_sum += entry['EO_1_0'].item()
        eo_1_0_count += 1
    if 'DP_0_0' in entry:
        dp_0_0_sum += entry['DP_0_0'].item()
        dp_0_0_count += 1
    if 'EO_0_0' in entry:
        eo_0_0_sum += entry['EO_0_0'].item()
        eo_0_0_count += 1


# Calculate averages
dp_0_0_avg = dp_0_0_sum / dp_0_0_count if dp_0_0_count > 0 else 0
eo_0_0_avg = eo_0_0_sum / eo_0_0_count if eo_0_0_count > 0 else 0

dp_0_1_avg = dp_0_1_sum / dp_0_1_count if dp_0_1_count > 0 else 0
eo_0_1_avg = eo_0_1_sum / eo_0_1_count if eo_0_1_count > 0 else 0
dp_1_0_avg = dp_1_0_sum / dp_1_0_count if dp_1_0_count > 0 else 0
eo_1_0_avg = eo_1_0_sum / eo_1_0_count if eo_1_0_count > 0 else 0

dp_0_1_avg, eo_0_1_avg, dp_1_0_avg, eo_1_0_avg, dp_0_0_avg, eo_0_0_avg

print("dp_0_1_avg",dp_0_1_avg)
print("eo_0_1_avg",eo_0_1_avg)

print("\ndp_1_0_avg",dp_1_0_avg)
print("eo_1_0_avg",eo_1_0_avg)

print("\ndp_0_0_avg",dp_0_0_avg)
print("eo_0_0_avg",eo_0_0_avg)

dp_0_1_avg 0.7809491625853947
eo_0_1_avg 0.823881688217322

dp_1_0_avg 0.9237945377826691
eo_1_0_avg 0.9560439586639404

dp_0_0_avg 0
eo_0_0_avg 1.0


In [17]:
# Normal

# dp_0_1_avg 0.739334006486864
# eo_0_1_avg 0.8355028525397584

# dp_1_0_avg 0.9306792974472046
# eo_1_0_avg 0.9176454544067383

# dp_0_0_avg 1.0
# eo_0_0_avg 1.0



# DP
# dp_0_1_avg 0.7358946942740192
# eo_0_1_avg 0.8404026189271141

# dp_1_0_avg 0.9261531972885132
# eo_1_0_avg 0.9098440254888227

# dp_0_0_avg 1.0
# eo_0_0_avg 1.0

# Fair

# dp_0_1_avg 0.6835256798832274
# eo_0_1_avg 0.8049088102688483

# dp_1_0_avg 0.9405220256132238
# eo_1_0_avg 0.9155816244773376

# dp_0_0_avg 1.0
# eo_0_0_avg 1.0


#DP Fair

# dp_0_1_avg 0.6811187832096542
# eo_0_1_avg 0.7972253190199716

# dp_1_0_avg 0.9153433680534363
# eo_1_0_avg 0.9016856407544699

# dp_0_0_avg 1.0
# eo_0_0_avg 1.0




In [28]:
temp_out[0]

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [45]:
temp_out

[tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 tensor([[1.],
         [1.],
      

In [20]:
temp_group[0]

tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1])

In [21]:
idx=15
num_zeros = torch.sum(temp_group[idx] == 0).item()
num_ones = torch.sum(temp_group[idx] == 1).item()

print(f"Number of 0s: {num_zeros}")
print(f"Number of 1s: {num_ones}")

temp_out[idx]

Number of 0s: 18
Number of 1s: 14


tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]])

[tensor([[0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]),
 tensor([[1.],
         [0.],
      

In [23]:
metric = BinaryFairness(2)

In [24]:
metric(temp_out[idx].view(-1), temp_lab[idx].view(-1), temp_group[idx]) 



{'DP_0_1': tensor(0.5983), 'EO_0_1': tensor(0.7143)}

In [34]:
# fairness_score

In [33]:
import torch
from torchmetrics.classification import BinaryFairness

preds = torch.tensor([1, 0, 1, 0, 1, 1, 1, 0])  # Predicted loan approvals
target = torch.tensor([1, 0, 1, 0, 0, 1, 0, 1])  # Actual loan approvals
groups = torch.tensor([0, 0, 1, 1, 0, 1, 1, 0])  # Group 0: male, Group 1: Female

fairness_metric = BinaryFairness(2)  # Calculate both demographic parity and equal opportunity

fairness_results = fairness_metric(preds, target, groups)

print("Demographic Parity:", fairness_results)




Demographic Parity: {'DP_0_1': tensor(0.6667), 'EO_0_1': tensor(0.5000)}


In [ ]:
client_data_dir="50_clients_data/processed_data/"

layout = [100, 100, 2]
batch_size = 32
num_epochs = 1
input_dim = 10
lr=0.001
noise_scale =0.1
model = FullyConnected(input_dim, layout)
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 